In [2]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler


# Carregar os dados
data = pd.read_csv('new_comentarios_ingles.csv')
df_1 = data['Comentario']
rotulos = data['sentimento']

# Definir polarizações de sentimento
def define_sentiment(rating):
    if rating == 'positivo':
        return 1
    elif rating == 'neutro':
        return 2
    else:
        return 0 

data['sentimento_polarizado'] = data['sentimento'].apply(define_sentiment)
rotulos = data['sentimento_polarizado']

# Tokenização e lematização
comentarios_tokenizados = []
wnl = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [wnl.lemmatize(plv) for plv in tokens]

for tok in df_1:
    tokens = nltk.word_tokenize(str(tok))
    comentarios_tokenizados.append(tokens)

lemm_tks = [lemmatize_tokens(tokens) for tokens in comentarios_tokenizados]

# Divisão em conjuntos de treinamento e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(lemm_tks, rotulos, test_size=0.2, random_state=42, stratify=rotulos)

# Vetorização dos dados de texto
vectorizer = CountVectorizer()
X_treino_vect = vectorizer.fit_transform([' '.join(comentario) for comentario in X_treino])
X_teste_vect = vectorizer.transform([' '.join(comentario) for comentario in X_teste])

# Aplicando Random Oversampling
oversampler = RandomOverSampler()
X_treino_res, y_treino_res = oversampler.fit_resample(X_treino_vect, y_treino)

# Treinamento do classificador Random Forest
classifier = MultinomialNB()

# Validação cruzada
scores = cross_val_score(classifier, X_treino_res, y_treino_res, cv=5)  # cv=5 indica 5-fold cross-validation

# Treinamento final do modelo
classifier.fit(X_treino_res, y_treino_res)

# Predição nos dados de teste
predicoes = classifier.predict(X_teste_vect)

# Calcular precisão, recall e F-score
precision = precision_score(y_teste, predicoes, average='weighted')
recall = recall_score(y_teste, predicoes, average='weighted')
f_score = f1_score(y_teste, predicoes, average='weighted')
acuracia = accuracy_score(y_teste, predicoes)
relatorio_classificacao = classification_report(y_teste, predicoes, target_names=['Negativo', 'Positivo', 'Neutro'])

# Imprimir resultados
print("Resultados da validação cruzada:")
print("Scores:", scores)
print("Precisão média:", scores.mean())
print("Acuracia: {:.2f}".format(acuracia))
print("Precisão: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F-score: {:.2f}".format(f_score))
print("\nRelatório de Classificação:")
print(relatorio_classificacao)


TypeError: __init__() got an unexpected keyword argument 'class_weight'

In [2]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN


# Carregar os dados
data = pd.read_csv('new_comentarios_ingles.csv')
df_1 = data['Comentario']
rotulos = data['sentimento']

df['Comentario'] = df['Comentario'].apply(lambda x: x.lower())

# Definir polarizações de sentimento
def define_sentiment(rating):
    if rating == 'positivo':
        return 1
    elif rating == 'neutro':
        return 2
    else:
        return 0 

data['sentimento_polarizado'] = data['sentimento'].apply(define_sentiment)
rotulos = data['sentimento_polarizado']

# Tokenização e lematização
comentarios_tokenizados = []
wnl = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [wnl.lemmatize(plv) for plv in tokens]

for tok in df_1:
    tokens = nltk.word_tokenize(str(tok))
    comentarios_tokenizados.append(tokens)

lemm_tks = [lemmatize_tokens(tokens) for tokens in comentarios_tokenizados]

# Divisão em conjuntos de treinamento e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(lemm_tks, rotulos, test_size=0.2, random_state=42, stratify=rotulos)

# Vetorização dos dados de texto
vectorizer = CountVectorizer()
X_treino_vect = vectorizer.fit_transform([' '.join(comentario) for comentario in X_treino])
X_teste_vect = vectorizer.transform([' '.join(comentario) for comentario in X_teste])


# Função para treinamento e avaliação do modelo
def train_and_evaluate_model(X_train_res, y_train_res, X_test_vect, y_test):
    classifier = MultinomialNB()

    # Validação cruzada
    scores = cross_val_score(classifier, X_train_res, y_train_res, cv=5)  # cv=5 indica 5-fold cross-validation

    # Treinamento final do modelo
    classifier.fit(X_train_res, y_train_res)

    # Predição nos dados de teste
    predicoes = classifier.predict(X_test_vect)

    # Calcular precisão, recall e F-score
    precision = precision_score(y_test, predicoes, average='weighted')
    recall = recall_score(y_test, predicoes, average='weighted')
    f_score = f1_score(y_test, predicoes, average='weighted')
    acuracia = accuracy_score(y_test, predicoes)
    relatorio_classificacao = classification_report(y_test, predicoes, target_names=['Negativo', 'Positivo', 'Neutro'])

    # Imprimir resultados
    print("Resultados da validação cruzada:")
    print("Scores:", scores)
    print("Precisão média:", scores.mean())
    print("Acuracia: {:.2f}".format(acuracia))
    print("Precisão: {:.2f}".format(precision))
    print("Recall: {:.2f}".format(recall))
    print("F-score: {:.2f}".format(f_score))
    print("\nRelatório de Classificação:")
    print(relatorio_classificacao)

# Random Undersampling
undersampler = RandomUnderSampler()
X_train_res, y_train_res = undersampler.fit_resample(X_treino_vect, y_treino)
print("\nRandom Undersampling:")
train_and_evaluate_model(X_train_res, y_train_res, X_teste_vect, y_teste)

# Random Undersampling
oversampler = RandomOverSampler()
X_train_res, y_train_res = oversampler.fit_resample(X_treino_vect, y_treino)
print("\nRandom oversampling:")
train_and_evaluate_model(X_train_res, y_train_res, X_teste_vect, y_teste)


# SMOTE
smote = SMOTE()
X_train_res, y_train_res = smote.fit_resample(X_treino_vect, y_treino)
print("\nSMOTE:")
train_and_evaluate_model(X_train_res, y_train_res, X_teste_vect, y_teste)

# ADASYN
adasyn = ADASYN()
X_train_res, y_train_res = adasyn.fit_resample(X_treino_vect, y_treino)
print("\nADASYN:")
train_and_evaluate_model(X_train_res, y_train_res, X_teste_vect, y_teste)

smote_enn = SMOTEENN()
X_treino_res, y_treino_res = smote_enn.fit_resample(X_treino_vect, y_treino)
print("\nBlack Transplantation (SMOTEENN):")
train_and_evaluate_model(X_treino_res, y_treino_res, X_teste_vect, y_teste)



Random Undersampling:
Resultados da validação cruzada:
Scores: [0.53119093 0.54820416 0.49716446 0.5        0.51325758]
Precisão média: 0.517963424414275
Acuracia: 0.56
Precisão: 0.84
Recall: 0.56
F-score: 0.65

Relatório de Classificação:
              precision    recall  f1-score   support

    Negativo       0.14      0.57      0.22       220
    Positivo       0.95      0.57      0.72      3268
      Neutro       0.15      0.43      0.22       317

    accuracy                           0.56      3805
   macro avg       0.41      0.52      0.39      3805
weighted avg       0.84      0.56      0.65      3805


Random oversampling:
Resultados da validação cruzada:
Scores: [0.81960734 0.80377407 0.80364656 0.81040418 0.8115517 ]
Precisão média: 0.8097967714562492
Acuracia: 0.68
Precisão: 0.82
Recall: 0.68
F-score: 0.74

Relatório de Classificação:
              precision    recall  f1-score   support

    Negativo       0.17      0.42      0.24       220
    Positivo       0.93     

In [5]:
pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
